In [29]:
import requests
import json

url_base = "http://localhost:8000"

In [30]:
endpoint = "/"

response = requests.get(url_base + endpoint)
print(response.json())

{'message': 'LLM as Judge Study API', 'version': '0.1.0', 'endpoints': {'compare': '/api/v1/compare', 'models': '/api/v1/models', 'health': '/api/v1/health', 'docs': '/docs'}}


In [31]:
endpoint = "/api/v1/compare"

data = {
    "input": "Qual a capital do Brasil?",
    "response_a": "A capital do Brasil é Brasília, localizada no Distrito Federal.",
    "response_b": "Brasília é a capital do Brasil desde 1960."
}

response = requests.post(url_base + endpoint, json=data, timeout=10)
print(response.json())

{'input': 'Qual a capital do Brasil?', 'response_a': 'A capital do Brasil é Brasília, localizada no Distrito Federal.', 'response_b': 'Brasília é a capital do Brasil desde 1960.', 'better_response': 'A', 'judge_reasoning': None, 'model_a_name': None, 'model_b_name': None, 'timestamp': '2025-08-24T18:36:01.393552Z', 'execution_time': 2.0644960403442383}


In [32]:
endpoint = "/api/v1/models"

response = requests.get(url_base + endpoint, timeout=10)
print(response.json())

{'available_models': ['llama-4-maverick', 'claude-4-sonnet', 'google-gemini-2.5-pro'], 'total_count': 3}


In [33]:
endpoint = "/api/v1/health"

response = requests.get(url_base + endpoint, timeout=10)
print(response.json())

{'status': 'healthy', 'timestamp': '2025-08-24T15:36:01.491625', 'service': 'LLM as Judge Study API', 'version': '0.1.0'}


In [34]:
# Definir configurações base
url_base = "http://localhost:8000"
endpoint = "/api/v1/compare/batch"

# Dados de teste para o endpoint batch
data_valid = {
    "comparisons": [
        {
            "input": "Qual a capital do Brasil?",
            "response_a": "A capital do Brasil é Brasília.",
            "response_b": "Brasília é a capital do Brasil desde 1960.",
            "model_a_name": "claude-4-sonnet",
            "model_b_name": "gemini-2.5-pro"
        },
        {
            "input": "Qual é o maior planeta do sistema solar?",
            "response_a": "O maior planeta do sistema solar é Júpiter.",
            "response_b": "Júpiter é o maior planeta, com massa maior que todos os outros planetas combinados.",
            "model_a_name": "llama-4-maverick",
            "model_b_name": "claude-4-sonnet"
        }
    ]
}

data_invalid = {
    "comparisons": [
        {"input": "Pergunta " + str(i), "response_a": f"Resposta A{i}", "response_b": f"Resposta B{i}"}
        for i in range(1, 7)  # 6 comparações - excede limite de 5
    ]
}

data_single = {
    "comparisons": [
        {
            "input": "Qual a capital do Brasil?",
            "response_a": "A capital do Brasil é Brasília.",
            "response_b": "Brasília é a capital do Brasil desde 1960."
        }
    ]
}

data_empty = {"comparisons": []}

data_no_payload = {}

def safe_request(method, url, **kwargs):
    """Faz request com tratamento seguro de erros"""
    try:
        response = getattr(requests, method)(url, **kwargs)
        return response
    except requests.RequestException as e:
        print(f"Erro de conexão: {e}")
        return None

def safe_json_response(response):
    """Extrai JSON com tratamento seguro de erros"""
    if response is None:
        return None
    
    try:
        return response.json()
    except ValueError:
        print(f"Erro ao decodificar JSON. Status: {response.status_code}")
        print(f"Raw response: {response.text[:200]}")
        return {"error": "Invalid JSON response", "status_code": response.status_code}

def print_response_info(response, test_name):
    """Imprime informações da resposta de forma padronizada"""
    print(f"\n=== {test_name} ===")
    
    if response is None:
        print("❌ Falha na conexão")
        return
    
    print(f"Status: {response.status_code}")
    
    # Verificar se resposta tem conteúdo
    if not response.text.strip():
        print("❌ Resposta vazia")
        return
        
    response_data = safe_json_response(response)
    
    if response_data:
        print("Response:", json.dumps(response_data, indent=2, ensure_ascii=False))
        
        # Destacar estatísticas de performance apenas para respostas 200
        if response.status_code == 200 and 'model_a_wins' in response_data:
            print("\n🏆 === ESTATÍSTICAS DE PERFORMANCE ===")
            print(f"📊 Modelo A (vitórias): {response_data['model_a_wins']}")
            print(f"📊 Modelo B (vitórias): {response_data['model_b_wins']}")
            print(f"📊 Empates: {response_data['ties']}")
            print(f"📊 Erros: {response_data['errors']}")
            print(f"🥇 Melhor modelo geral: {response_data['best_model']}")
            print(f"⏱️ Tempo de execução: {response_data['execution_time']:.2f}s")
    else:
        print("❌ Falha ao processar resposta JSON")

# Executar testes
print("🚀 Iniciando testes do endpoint batch...")

# TESTE 1: Request válido
response = safe_request("post", url_base + endpoint, json=data_valid, timeout=30)
print_response_info(response, "TESTE 1: Request válido (2 comparações)")

# TESTE 2: Request inválido (muitas comparações)
response = safe_request("post", url_base + endpoint, json=data_invalid, timeout=30)
print_response_info(response, "TESTE 2: Request inválido (6 comparações - limite é 5)")

# TESTE 3: Request com apenas 1 comparação (deve sugerir endpoint individual)
response = safe_request("post", url_base + endpoint, json=data_single, timeout=30)
print_response_info(response, "TESTE 3: Apenas 1 comparação (deve sugerir endpoint individual)")

# TESTE 4: Lista vazia
response = safe_request("post", url_base + endpoint, json=data_empty, timeout=30)
print_response_info(response, "TESTE 4: Lista vazia (deve dar erro - mínimo 2)")

# TESTE 5: Sem payload
response = safe_request("post", url_base + endpoint, json=data_no_payload, timeout=30)
print_response_info(response, "TESTE 5: Sem payload (deve dar erro - campo obrigatório)")

# TESTE 6: Sem JSON
response = safe_request("post", url_base + endpoint, timeout=30)
print_response_info(response, "TESTE 6: Sem JSON (deve dar erro - body vazio)")

print("\n✅ Testes concluídos!")

🚀 Iniciando testes do endpoint batch...

=== TESTE 1: Request válido (2 comparações) ===
Status: 200
Response: {
  "results": [
    {
      "id": "074b74a4-dbe6-491d-bf57-28fef5df2731",
      "input": "Qual a capital do Brasil?",
      "response_a": "A capital do Brasil é Brasília.",
      "response_b": "Brasília é a capital do Brasil desde 1960.",
      "model_a_name": "claude-4-sonnet",
      "model_b_name": "gemini-2.5-pro",
      "better_response": "B",
      "judge_reasoning": null
    },
    {
      "id": "718a3b82-aa9b-4753-aa05-7d81969f28e0",
      "input": "Qual é o maior planeta do sistema solar?",
      "response_a": "O maior planeta do sistema solar é Júpiter.",
      "response_b": "Júpiter é o maior planeta, com massa maior que todos os outros planetas combinados.",
      "model_a_name": "llama-4-maverick",
      "model_b_name": "claude-4-sonnet",
      "better_response": "A",
      "judge_reasoning": null
    }
  ],
  "total_comparisons": 2,
  "successful": 2,
  "executio